 ### Cell below contains all import code

In [243]:
import matplotlib.image as mpimg
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2

### Function definitions go here

#### Reading an image

In [244]:
def readImg(path):
    image = mpimg.imread(path)
    imagexsize = image.shape[1]
    imageysize = image.shape[0]
    #Returns an array containing image array as first element and image dimension as second element. Image dimensions is 
    #an array given by [ysize, xsize]
    return [image, image.shape]
#plt.imshow(readImg(collectImages("test_images")[0])[0])
#plt.show()

#### Collecting image names from a folder

In [245]:
def collectImages(folderPath):
    # Returns array of strings with complete path of a file from current folder.
    return [folderPath + imageName for imageName in os.listdir(folderPath + "/")]
#print (collectImages("test_images"))

#### Convert RGB image to Gray Scale

In [246]:
def convertToGrayScale (rgbImage):
    grayImage = np.copy(rgbImage)
    # Returns 8 bit image array converted from input 32-bit (8+8+8) RGB image.
    return cv2.cvtColor(grayImage, cv2.COLOR_RGB2GRAY)
#plt.imshow(convertToGrayScale(readImg(collectImages("test_images")[0])[0]), cmap="gray")
#plt.show()

#### Apply Gaussian Blur

In [247]:
def applyGaussianBlur (image, kernelSize=3):
    filteredImage = np.copy(image)
    # Return image array after applying Gaussian blur. Image array size depends on input image.
    return cv2.GaussianBlur(filteredImage, (kernelSize, kernelSize), 0)
#plt.imshow(convertToGrayScale(readImg(collectImages("test_images")[0])[0]), cmap="gray")
#plt.show()
#plt.imshow(applyGaussianBlur(convertToGrayScale(readImg(collectImages("test_images")[0])[0]), 15), cmap="gray")
#plt.show()

#### Apply Canny Edge Detection

In [248]:
def cannyEdgeDetector (grayImage, lowThreshold=1, highThreshold=10):
    edges = cv2.Canny(grayImage, lowThreshold, highThreshold)
    # Returns 8-bit image array of size same as input gray scale image with boolean True for edges and False for non-edges
    return edges
#plt.imshow(convertToGrayScale(readImg(collectImages("test_images")[2])[0]), cmap="gray")
#plt.show()
#plt.imshow(cannyEdgeDetector(applyGaussianBlur(convertToGrayScale(readImg(collectImages("test_images")[2])[0]), 5), 50, 200), cmap="gray")
#plt.show()

#### Filter out region of Interest

In [249]:
def createPolyFilter (vertices):
    vertexArr = []
    for vertex in vertices:
        vertexArr.append((vertex[0], vertex[1]))
    vertexArr = np.array([vertexArr], dtype=np.int32)
    # Returns array of vertices as ordered pair tuples with data type as 32bit float
    return vertexArr
#print (createPolyFilter([(0, 0), (3,2), (400,322)]))

In [250]:
def applyRegionFilter (image, verticeArray):
    mask = np.zeros_like(image)
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
    #Create polygon and fill pixels inside polgon with ignore_mask_color
    cv2.fillPoly(mask, verticeArray, ignore_mask_color)
    maskedImage = cv2.bitwise_and(image, mask)
    # Returns image of same size as input image after applying region of interest filter
    return maskedImage
#plt.imshow(cannyEdgeDetector(applyGaussianBlur(convertToGrayScale(readImg(collectImages("test_images")[2])[0]), 5), 50, 200), cmap="gray")
#plt.show()
#imagexsize = readImg(collectImages("test_images")[1])[1][1]
#imageysize = readImg(collectImages("test_images")[1])[1][0]
#plt.imshow(applyRegionFilter(cannyEdgeDetector(applyGaussianBlur(convertToGrayScale(readImg(collectImages("test_images")[2])[0]), 5), 50, 200), \
#                           createPolyFilter([(int(imagexsize/10), imageysize),\
#                            (int(imagexsize/2), int(11*imageysize/20)),\
#                            (int(6*imagexsize/10), int(11*imageysize/20)),\
#                           (int(9*imagexsize/10), imageysize)])),\
#          cmap="gray")
#plt.show()

#### Draw Hough lines

In [263]:
def drawHoughLines (image, edgeImage, rho=1, theta=(np.pi/180), threshold=1, minLineLength=2, maxLineGap=5):
    houghLines = cv2.HoughLinesP(edgeImage, rho, theta, threshold, np.array([]), minLineLength, maxLineGap)
    lineImage = np.copy(image)
    for line in houghLines:
        for x1, y1, x2, y2 in line:
            cv2.line(lineImage, (x1,y1), (x2,y2), (255,0,0), 5)
    # Returns original image with lines detected by Hough superimposed
    return lineImage
#image = readImg(collectImages("test_images")[5])
#imagexsize = image[1][1]
#imageysize = image[1][0]
#edgeImage = applyRegionFilter(cannyEdgeDetector(applyGaussianBlur(convertToGrayScale(image[0]), 5), 50, 240), \
#                           createPolyFilter([(int(imagexsize/20), imageysize),\
#                            (int(imagexsize/2), int(12*imageysize/20)),\
#                            (int(6*imagexsize/10), int(12*imageysize/20)),\
#                           (int(18*imagexsize/20), imageysize)]))
#plt.imshow(edgeImage, cmap="gray")
#plt.show()
#plt.imshow(drawHoughLines(image[0], edgeImage, threshold=20, minLineLength=20, maxLineGap=80))
#plt.show()


### Main method to test code

In [266]:
def main():
    pathToImageFolder = "test_images/"
    
    for imageName in collectImages(pathToImageFolder):
        
        imageWithAttr = readImg(imageName)
        imagexsize = imageWithAttr[1][1]
        imageysize = imageWithAttr[1][0]
        kernelSize = 5
        cannyLowThreshold = 50
        cannyHighThreshold = 240
        regionPolyVector = createPolyFilter([(int(imagexsize/20), imageysize),\
                                (int(imagexsize/2), int(12*imageysize/20)),\
                                (int(6*imagexsize/10), int(12*imageysize/20)),\
                               (int(18*imagexsize/20), imageysize)])

        houghThreshold = 20
        houghMinLineLenght = 20
        houghMaxLineGap = 80

        grayScaledImage = convertToGrayScale(imageWithAttr[0])
        gaussianBlurredImage = applyGaussianBlur(grayScaledImage, kernelSize)
        edgeImage = cannyEdgeDetector(gaussianBlurredImage, cannyLowThreshold, cannyHighThreshold)
        regionSelectedImage = applyRegionFilter(edgeImage, regionPolyVector)
        imageWithHoughLines = drawHoughLines(imageWithAttr[0], regionSelectedImage, threshold=houghThreshold,\
                                             minLineLength=houghMinLineLenght, maxLineGap=houghMaxLineGap)
        #plt.imshow(imageWithHoughLines)
        #plt.show()
        #mpimg.imsave(imageName[0:imageName.rindex(".")] + "_lane_detected" + imageName[imageName.rindex("."):], imageWithHoughLines)
main()